# Prompting e Output Estruturado

A qualidade das respostas de um modelo de linguagem depende diretamente da qualidade das instrucoes que ele recebe. Esse processo de formular instrucoes e chamado de **prompting**.

Neste notebook vamos explorar diferentes tecnicas de prompting, desde instrucoes simples ate a extracao de dados estruturados com validacao de tipos usando Pydantic.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Prompting basico

Quando criamos um agente sem nenhuma instrucao adicional, ele se comporta como um assistente generico. A resposta vai depender exclusivamente do treinamento do modelo.

In [2]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage

agente = create_agent(model="gpt-4.1-nano")

In [3]:
pergunta = HumanMessage(content="Analise a empresa Nubank.")

resposta = agente.invoke(
    {"messages": [pergunta]}
)

print(resposta["messages"][-1].content)

Claro, aqui está uma análise da Nubank com base nas informações disponíveis até outubro de 2023:

**Visão Geral da Nubank**  
A Nubank é uma fintech brasileira fundada em 2013, que se destacou por oferecer uma proposta de banco digital totalmente digital, com foco na simplicidade, transparência e valorização da experiência do cliente. Sua sede fica em São Paulo, Brasil, e ela opera principalmente na América Latina, com presença significativa no Brasil, além de expandir para outros países como México e Colômbia.

**Principais Produtos e Serviços**  
- **Cartão de Crédito sem anuidade:** Programa de cartões com gestão via aplicativo, tecnologia de pagamento por aproximação, sem tarifas de anuidade.
- **Conta Digital:** Conta totalmente digital, que permite movimentações financeiras, transferências, pagamentos e outros serviços bancários.
- **Empréstimos e Créditos:** Oferece empréstimos pessoais, crédito para pequenas empresas e produtos de antecipação de recebíveis.
- **Investimentos:**

A resposta e generica porque o modelo nao sabe qual o papel dele na conversa. Podemos mudar completamente o comportamento do agente adicionando um **system prompt**, que define a persona e as instrucoes que o modelo deve seguir.

In [4]:
system_prompt = "Voce e um analista de negocios especializado em startups brasileiras. Seja direto e objetivo nas analises."

analista = create_agent(
    model="gpt-4.1-nano",
    system_prompt=system_prompt
)

In [5]:
resposta = analista.invoke(
    {"messages": [pergunta]}
)

print(resposta["messages"][-1].content)

O Nubank é uma fintech brasileira de destaque, reconhecida por sua proposta inovadora no setor financeiro. Seus principais pontos positivos incluem alto crescimento de clientes, forte valorização de marca e inovação em produtos digitais. Diversificou seus serviços, ampliando a oferta de cartões, contas digitais, empréstimos e seguros.

Desafios: 
- Baixa rentabilidade devido a altas despesas com aquisição e operações.
- Competição crescente de bancos tradicionais e novas fintechs.
- Regulamentação e risco de crédito.

Potencial: Ainda há espaço para crescimento, especialmente na expansão de produtos e no mercado de crédito adulto no Brasil, além de possibilidades internacionais. A estratégia de foco em experiência do usuário e inovação tecnológica sustenta sua competitividade futura.


Observe como a mesma pergunta gerou uma resposta completamente diferente. O system prompt e a forma mais direta de controlar o comportamento do modelo.

## Few-shot prompting

Em muitos casos, descrever o que voce quer nao e suficiente. E mais eficaz **mostrar exemplos** do formato esperado. Essa tecnica e chamada de **few-shot prompting**: incluimos alguns pares de entrada e saida no system prompt para que o modelo aprenda o padrao desejado.

In [6]:
system_prompt = """
Voce e um analista de negocios especializado em startups brasileiras.
Responda com analises curtas e diretas.

Usuario: Analise a empresa iFood.
Analista: iFood domina o delivery de comida no Brasil com mais de 80% de market share. Modelo de marketplace com receita via comissoes e assinaturas. Desafio principal: manter margens com alto custo de logistica.

Usuario: Analise a empresa QuintoAndar.
Analista: QuintoAndar revolucionou o aluguel de imoveis ao eliminar fiador e burocracias. Modelo de plataforma com receita via taxas de intermediacao. Expandiu para compra e venda, aumentando o mercado enderecavel.
"""

analista = create_agent(
    model="gpt-4.1-nano",
    system_prompt=system_prompt
)

In [7]:
resposta = analista.invoke(
    {"messages": [pergunta]}
)

print(resposta["messages"][-1].content)

Nubank é uma fintech líder no Brasil, reconhecida por oferecer cartões de crédito e serviços bancários digitais sem tarifas tradicionais. Modelo de plataforma com receita via tarifas de intercâmbio, anuidades e juros de crédito. Forte crescimento impulsionado por inovação, facilidade de uso e expansão internacional na América Latina. Desafio principal: manter a rentabilidade diante de alta concorrência e custos operacionais crescentes.


Com os exemplos, o modelo entendeu o **tom**, o **tamanho** e a **estrutura** que queremos. Isso e muito mais eficiente do que tentar descrever o formato com palavras.

## Prompt estruturado

Outra abordagem e definir explicitamente os **campos** que a resposta deve conter. Isso funciona como um template que o modelo preenche. E util quando voce precisa de consistencia entre multiplas chamadas.

In [8]:
system_prompt = """
Voce e um analista de negocios especializado em startups brasileiras.

Para cada empresa analisada, responda exatamente no formato abaixo:

Empresa: nome da empresa
Setor: setor de atuacao
Modelo de negocio: como a empresa gera receita
Diferencial: principal vantagem competitiva
Risco: principal risco ou desafio
"""

analista = create_agent(
    model="gpt-4.1-nano",
    system_prompt=system_prompt
)

In [9]:
resposta = analista.invoke(
    {"messages": [pergunta]}
)

print(resposta["messages"][-1].content)

Empresa: Nubank  
Setor: Fintech / Serviços Financeiros  
Modelo de negocio: Gera receita principalmente através de tarifas de anuidade de cartões de crédito, juros de empréstimos pessoais e financiamentos, além de cobrar por serviços adicionais como seguros e serviços financeiros diversos.  
Diferencial: Oferece uma experiência totalmente digital, com um modelo de operação enxuto, transparência nas tarifas e atendimento ao cliente via aplicativo, além de uma abordagem inovadora ao desafiar os bancos tradicionais com uma proposta mais acessível e centrada no usuário.  
Risco: Alta concorrência no setor financeiro digital e dependência de financiamento de crescimento através de captação de recursos no mercado de capitais.


In [10]:
resposta = analista.invoke(
    {"messages": [HumanMessage(content="Analise a empresa Stone.")]}
)

print(resposta["messages"][-1].content)

Empresa: Stone  
Setor: Fintech / Serviços de pagamento  
Modelo de negocio: A Stone gera receita principalmente através de taxas cobradas por suas soluções de pagamento, incluindo máquinas de cartão de crédito e débito, além de serviços financeiros complementares como empréstimos e gerenciamento de câmbio para empresas.  
Diferencial: Seu principal diferencial está na tecnologia de ponto de venda (POS) inovadora, atendimento personalizado ao cliente e forte atuação em pequenas e médias empresas, além de uma estratégia de relacionamento que promove fidelização.  
Risco: O principal risco é a crescente concorrência no setor de pagamentos digitais, tanto de empresas tradicionais quanto de novas startups, além de possíveis mudanças regulatórias e riscos de expansão internacional.


O formato e consistente entre chamadas diferentes, o que e importante para pipelines automatizados. Porem, a resposta ainda e texto puro. Se quisermos usar esses dados em codigo (salvar em banco, comparar empresas, gerar relatorios), precisamos de algo mais robusto.

## Structured output

O LangChain permite forcar o modelo a retornar dados em um formato **estruturado e tipado**, usando modelos do Pydantic. Em vez de receber texto livre, recebemos um objeto Python com campos validados.

Isso e fundamental para integrar LLMs em sistemas reais, onde os dados precisam ser consumidos por outras partes do codigo.

In [11]:
from pydantic import BaseModel

class AnaliseEmpresa(BaseModel):
    empresa: str
    setor: str
    modelo_de_negocio: str
    diferencial: str
    risco: str

In [12]:
analista = create_agent(
    model="gpt-4.1-nano",
    system_prompt="Voce e um analista de negocios especializado em startups brasileiras.",
    response_format=AnaliseEmpresa
)

In [13]:
resposta = analista.invoke(
    {"messages": [pergunta]}
)

resposta["structured_response"]

AnaliseEmpresa(empresa='Nubank', setor='Finanças / Serviços Financeiros', modelo_de_negocio='Plataforma digital de cartões de crédito e serviços bancários sem tarifas tradicionais, com foco em experiência do cliente, inovação tecnológica e onboarding digital eficiente.', diferencial='Foco na experiência do usuário, simplicidade na gestão financeira, ausência de tarifas em muitos serviços, tecnologia de ponta e forte branding digital.', risco='Alta competitividade no setor financeiro, riscos regulatórios, vulnerabilidade a mudanças tecnológicas e econômicas, além da necessidade de constante inovação para manter a liderança.')

O retorno agora e um objeto `AnaliseEmpresa`, nao texto. Podemos acessar cada campo diretamente como atributo Python.

In [14]:
analise = resposta["structured_response"]

print(type(analise))

<class '__main__.AnaliseEmpresa'>


In [15]:
print(analise.empresa)
print(analise.setor)
print(analise.diferencial)

Nubank
Finanças / Serviços Financeiros
Foco na experiência do usuário, simplicidade na gestão financeira, ausência de tarifas em muitos serviços, tecnologia de ponta e forte branding digital.


In [16]:
print(f"{analise.empresa} atua no setor de {analise.setor}.")
print(f"Diferencial: {analise.diferencial}")
print(f"Risco: {analise.risco}")

Nubank atua no setor de Finanças / Serviços Financeiros.
Diferencial: Foco na experiência do usuário, simplicidade na gestão financeira, ausência de tarifas em muitos serviços, tecnologia de ponta e forte branding digital.
Risco: Alta competitividade no setor financeiro, riscos regulatórios, vulnerabilidade a mudanças tecnológicas e econômicas, além da necessidade de constante inovação para manter a liderança.


Essa abordagem transforma o LLM em uma **funcao que retorna dados tipados**, o que permite integrar facilmente com bancos de dados, APIs, dashboards ou qualquer outro sistema. A validacao dos tipos e feita automaticamente pelo Pydantic, garantindo que os dados estejam no formato esperado.